# Get all old questions

In [ ]:
import load_secrets
load_secrets.load_secrets()

In [ ]:
from forecasting_tools import MetaculusApi, ApiFilter, DataOrganizer
from datetime import datetime
import asyncio, time, joblib

api_filter = ApiFilter(
    allowed_statuses=["resolved"],
)

all_resolved = []

for chunk in range(0,1000):
    offset = chunk * MetaculusApi.MAX_QUESTIONS_FROM_QUESTION_API_PER_REQUEST
    questions, success = MetaculusApi._grab_filtered_questions_with_offset(api_filter, offset)
    if success:
        all_resolved.extend(questions)
        print(len(all_resolved))
        time.sleep(1.1)
    else:
        print('done')
        break

joblib.dump(all_resolved, 'all_resolved.joblib')

In [43]:
questions = joblib.load('all_resolved.joblib')

In [44]:
from flatten_dict import flatten_dict

In [46]:
vars(questions[100]).keys()

dict_keys(['question_text', 'id_of_post', 'page_url', 'id_of_question', 'state', 'num_forecasters', 'num_predictions', 'resolution_criteria', 'fine_print', 'background_info', 'unit_of_measure', 'close_time', 'actual_resolution_time', 'scheduled_resolution_time', 'published_time', 'open_time', 'date_accessed', 'already_forecasted', 'tournament_slugs', 'includes_bots_in_aggregates', 'cp_reveal_time', 'api_json', 'upper_bound', 'lower_bound', 'open_upper_bound', 'open_lower_bound', 'zero_point'])

In [48]:
import pandas as pd
df = pd.DataFrame([flatten_dict(vars(q)) for q in questions])

In [78]:
rezzies = sorted(df['api_json.question.resolution'].unique())

In [82]:
df1 = df[df['api_json.question.resolution'].apply(lambda x: x not in ['ambiguous', 'annulled'])]

In [84]:
df2 = df1[df1['num_forecasters'] > 40]

In [85]:
len(df2)

2840

In [86]:
df2

,question_text,id_of_post,page_url,id_of_question,state,num_forecasters,num_predictions,resolution_criteria,fine_print,background_info,...,api_json.projects.topic,api_json.question.aggregations.metaculus_prediction.latest.start_time,api_json.question.aggregations.metaculus_prediction.latest.end_time,api_json.question.aggregations.metaculus_prediction.latest.forecast_values,api_json.question.aggregations.metaculus_prediction.latest.forecaster_count,api_json.question.aggregations.metaculus_prediction.latest.interval_lower_bounds,api_json.question.aggregations.metaculus_prediction.latest.centers,api_json.question.aggregations.metaculus_prediction.latest.interval_upper_bounds,api_json.question.aggregations.metaculus_prediction.latest.means,api_json.question.aggregations.metaculus_prediction.latest.histogram
3,What will be the reported value of the S&P/Cas...,37793,https://www.metaculus.com/questions/37793,37115,QuestionState.RESOLVED,76,81,This question resolves as the reported value b...,,Business Insider: [Housing market forecast: 20...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,How much in revenues will Wix report for Q1 2025?,37615,https://www.metaculus.com/questions/37615,36968,QuestionState.RESOLVED,44,47,This question resolves as the total revenues r...,,[Wix to Announce First Quarter 2025 Results on...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Will Mountain View, Colorado town councilor Br...",37614,https://www.metaculus.com/questions/37614,36967,QuestionState.RESOLVED,77,81,This question resolves as **Yes** if Brent Met...,,[Town councilor accused of shooting teen in fa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Will the USDA-posted recall by Johnsonville LL...,37475,https://www.metaculus.com/questions/37475,36835,QuestionState.RESOLVED,75,79,This question resolves as **Yes** if the statu...,,"Real Simple: [Over 22,000 Pounds of a Popular ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Will Berkshire Hathaway reveal a purchase of m...,37463,https://www.metaculus.com/questions/37463,36823,QuestionState.RESOLVED,77,80,This question resolves as **Yes** if the 13F f...,,StockCircle: [Berkshire Hathaway's Amazon Stak...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6124,Will the cost of sequencing a human genome fal...,7,https://www.metaculus.com/questions/7,7,QuestionState.RESOLVED,79,96,The cost of sequencing a human genome has been...,,,...,NaN,1.455433e+09,NaN,"[0.7841228415907086, 0.21587715840929148]",78.0,None,None,None,NaN,NaN
6126,Is Graph Isomorphism solvable in Polynomial Time?,16,https://www.metaculus.com/questions/16,16,QuestionState.RESOLVED,78,101,"In graph theory, two graphs are isomorphic if ...",,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6130,Will AI systems that can generate a synopsis o...,9,https://www.metaculus.com/questions/9,9,QuestionState.RESOLVED,55,63,Will a working version of such a system be pub...,,In late 2014 Google [announced] a deep learnin...,...,"[{'id': 15869, 'name': 'Artificial Intelligenc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6131,Will the Twin Prime Conjecture be positively r...,8,https://www.metaculus.com/questions/8,8,QuestionState.RESOLVED,124,166,Conjecture: There are infinitely many primes *...,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
!mkdir resolved

In [95]:
question_ids = df2.id_of_question.values

In [96]:
from tqdm import tqdm

In [97]:
for question in tqdm(questions):
    if question.id_of_question in question_ids:
        fn=f'resolved/{question.id_of_question}.joblib'
        joblib.dump(question, fn)


100%|██████████████████████████████████████| 6134/6134 [00:12<00:00, 480.23it/s]
